In [135]:
import math
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import tensorflow as tf

from sklearn import metrics
from sklearn.utils import shuffle

In [136]:
housing_df = pd.read_csv("../datasets/housing-prices-dataset/train.csv")

In [137]:
housing_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [138]:
processed_features = housing_df[["GrLivArea"]]
output_targets = pd.DataFrame(housing_df["SalePrice"])

In [139]:
#splitting data 
training_examples = processed_features[0:1000]
training_targets = output_targets[0:1000]

val_examples = processed_features[1001:1200]
val_targets = output_targets[1001:1200]

test_examples = processed_features[1201:1460]
test_targets = output_targets[1201:1460]

In [140]:
my_feature_columns = [tf.feature_column.numeric_column("GrLivArea")]

my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

In [141]:
model = tf.estimator.DNNRegressor(feature_columns=my_feature_columns,
                                  hidden_units=[20,20],
                                  optimizer=my_optimizer)

In [142]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):

    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    ds = Dataset.from_tensor_slices((features,targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
      
    features, labels = ds.make_one_shot_iterator().get_next()
    
    return features, labels

In [143]:
training = model.train(input_fn = lambda:my_input_fn(training_examples,training_targets["SalePrice"],
                                                     batch_size=32),steps=2000)

In [144]:
train_predictions = model.predict(input_fn=lambda: my_input_fn(training_examples, training_targets, num_epochs=1, shuffle=False))
val_predictions = model.predict(input_fn=lambda: my_input_fn(val_examples, val_targets, num_epochs=1, shuffle=False))
test_predictions = model.predict(input_fn=lambda: my_input_fn(test_examples, test_targets, num_epochs=1, shuffle=False))

In [145]:
train_predictions = np.array([item['predictions'][0] for item in train_predictions])
val_predictions = np.array([item['predictions'][0] for item in val_predictions])
test_predictions = np.array([item['predictions'][0] for item in test_predictions])

In [146]:
mean_squared_error = metrics.mean_squared_error(train_predictions, training_targets)
root_mean_squared_error = math.sqrt(mean_squared_error)

mean_squared_error = metrics.mean_squared_error(val_predictions, val_targets)
root_mean_squared_error = math.sqrt(mean_squared_error)

mean_squared_error = metrics.mean_squared_error(test_predictions, test_targets)
root_mean_squared_error = math.sqrt(mean_squared_error)

In [147]:
print('root mean squared error (RMSE) on VALIDATION data: {:0.3f}'.format(root_mean_squared_error))
print("root mean squared error (RMSE) on TRAINING data: {:0.3f}".format(root_mean_squared_error))
print('root mean squared error (RMSE) on TESTING data: {:0.3f}'.format(root_mean_squared_error))

root mean squared error (RMSE) on VALIDATION data: 66794.443
root mean squared error (RMSE) on TRAINING data: 66794.443
root mean squared error (RMSE) on TESTING data: 66794.443
